
##Setup

You will need to make a copy of this Colab notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.


In [1]:
import os
from google_drive_downloader import GoogleDriveDownloader as gdd

# Need to download the Omniglot dataset -- DON'T MODIFY THIS CELL
if not os.path.isdir('./omniglot_resized'):
    gdd.download_file_from_google_drive(file_id='1iaSFXIYC3AB8q9K_M-oVMa4pmB7yKMtI',
                                        dest_path='./omniglot_resized.zip',
                                        unzip=True)
    
assert os.path.isdir('./omniglot_resized')

In [2]:
import numpy as np
import os
import random
import tensorflow as tf
from scipy import misc

def get_images(paths, labels, nb_samples=None, shuffle=True):
    """
    Takes a set of character folders and labels and returns paths to image files
    paired with labels.
    Args:
        paths: A list of character folders
        labels: List or numpy array of same length as paths
        nb_samples: Number of images to retrieve per character
    Returns:
        List of (label, image_path) tuples
    """
    if nb_samples is not None:
        sampler = lambda x: random.sample(x, nb_samples)
    else:
        sampler = lambda x: x
    images_labels = [(i, os.path.join(path, image))
                     for i, path in zip(labels, paths)
                     for image in sampler(os.listdir(path))]
    if shuffle:
        random.shuffle(images_labels)
    return images_labels


def image_file_to_array(filename, dim_input):
    """
    Takes an image path and returns numpy array
    Args:
        filename: Image filename
        dim_input: Flattened shape of image
    Returns:
        1 channel image
    """
    import imageio
    image = imageio.imread(filename)  # misc.imread(filename)
    image = image.reshape([dim_input])
    image = image.astype(np.float32) / 255.0
    image = 1.0 - image
    return image


class DataGenerator(object):
    """
    Data Generator capable of generating batches of Omniglot data.
    A "class" is considered a class of omniglot digits.
    """

    def __init__(self, num_classes, num_samples_per_class, config={}):
        """
        Args:
            num_classes: Number of classes for classification (K-way)
            num_samples_per_class: num samples to generate per class in one batch
            batch_size: size of meta batch size (e.g. number of functions)
        """
        self.num_samples_per_class = num_samples_per_class
        self.num_classes = num_classes

        data_folder = config.get('data_folder', './omniglot_resized')
        self.img_size = config.get('img_size', (28, 28))

        self.dim_input = np.prod(self.img_size)
        self.dim_output = self.num_classes

        character_folders = [os.path.join(data_folder, family, character)
                             for family in os.listdir(data_folder)
                             if os.path.isdir(os.path.join(data_folder, family))
                             for character in os.listdir(os.path.join(data_folder, family))
                             if os.path.isdir(os.path.join(data_folder, family, character))]

        random.seed(1)
        random.shuffle(character_folders)
        num_val = 100
        num_train = 1100
        self.metatrain_character_folders = character_folders[: num_train]
        self.metaval_character_folders = character_folders[
            num_train:num_train + num_val]
        self.metatest_character_folders = character_folders[
            num_train + num_val:]

    def sample_batch(self, batch_type, batch_size):
        """
        Samples a batch for training, validation, or testing
        Args:
            batch_type: train/val/test
        Returns:
            A a tuple of (1) Image batch and (2) Label batch where
            image batch has shape [B, K, N, 784] and label batch has shape [B, K, N, N]
            where B is batch size, K is number of samples per class, N is number of classes
        """
        if batch_type == "train":
            folders = self.metatrain_character_folders
        elif batch_type == "val":
            folders = self.metaval_character_folders
        else:
            folders = self.metatest_character_folders

        #############################
        #### YOUR CODE GOES HERE ####
        all_image_batches = np.zeros(shape=(batch_size, self.num_samples_per_class, self.num_classes, self.dim_input), dtype=np.float32)
        all_label_batches = np.zeros(shape=(batch_size, self.num_samples_per_class, self.num_classes, self.num_classes), dtype=np.float32)
        

        for batch_id in range(batch_size):
            tmp = np.random.randint(0, self.num_classes)
            paths = np.random.choice(folders, self.num_classes)
            one_hot_labels = np.identity(self.num_classes)
            if batch_type == "test":
                np.random.shuffle(one_hot_labels)
                
            images_labels = get_images(paths, range(self.num_classes), nb_samples=self.num_samples_per_class, shuffle=False)
            
            sam_per_class = {}
            for i in range(self.num_classes):
                sam_per_class[i] = 0

            for idx, (label, fname) in enumerate(images_labels):                  
                img = image_file_to_array(fname, self.dim_input)
                all_image_batches[batch_id, sam_per_class[label], label, :] = img
                all_label_batches[batch_id, sam_per_class[label], label, :] = one_hot_labels[:, label]
                sam_per_class.update({label:sam_per_class[label]+1})

            
        #############################

        return all_image_batches.astype(np.float32), all_label_batches.astype(np.float32)

In [4]:
num_classes = 4
num_samples = 2
data_generator = DataGenerator(num_classes, num_samples + 1)
i, l = data_generator.sample_batch('test', 100)
i.shape, l.shape

l[0]

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)

In [5]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers

class MANN(tf.keras.Model):

    def __init__(self, num_classes, samples_per_class):
        super(MANN, self).__init__()
        self.num_classes = num_classes
        self.samples_per_class = samples_per_class
        self.layer1 = tf.keras.layers.LSTM(128, return_sequences=True)
        self.layer2 = tf.keras.layers.LSTM(num_classes, return_sequences=True)

    def call(self, input_images, input_labels):
        """
        MANN
        Args:
            input_images: [B, K+1, N, 784] flattened images
            labels: [B, K+1, N, N] ground truth labels
        Returns:
            [B, K+1, N, N] predictions
        """
        #############################
        #### YOUR CODE GOES HERE ####
        #print('anu')
        B, K, N, D = input_images.shape
        images = tf.reshape(input_images, (B, K*N, D))
        
        # filling the last N examples with zeros
        labels = tf.concat((input_labels[:,:-1], tf.zeros_like(input_labels[:, -1:])), axis=1)
        
        # reshaping to match with images
        labels = tf.reshape(labels, (B, K*N, N))

        tmp = self.layer1(tf.concat((images, labels), axis=-1))
        tmp = self.layer2(tmp)
        out = tf.reshape(tmp, (B, K, N, N))
        #############################
        return out

    def loss_function(self, preds, labels):
        """
        Computes MANN loss
        Args:
            preds: [B, K+1, N, N] network output
            labels: [B, K+1, N, N] labels
        Returns:
            scalar loss
        """
        #############################
        #### YOUR CODE GOES HERE ####
        loss = tf.reduce_sum(tf.keras.losses.categorical_crossentropy(y_true=labels[:, -1, :, :], y_pred=preds[:, -1, :, :], from_logits=True))
        #############################
        return loss

@tf.function
def train_step(images, labels, model, optim, eval=False):
    with tf.GradientTape() as tape:
        predictions = model(images, labels)
        loss = model.loss_function(predictions, labels)
    if not eval:
        gradients = tape.gradient(loss, model.trainable_variables)
        optim.apply_gradients(zip(gradients, model.trainable_variables))
    return predictions, loss


def main(num_classes=5, num_samples=1, meta_batch_size=16, random_seed=1234):
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    data_generator = DataGenerator(num_classes, num_samples + 1)

    o = MANN(num_classes, num_samples + 1)
    optim = tf.keras.optimizers.Adam(learning_rate=0.001)

    result = dict()
    
    for step in range(25000):
        i, l = data_generator.sample_batch('train', meta_batch_size)
        _, ls = train_step(i, l, o, optim)

        if (step + 1) % 100 == 0:
            print("*" * 5 + "Iter " + str(step + 1) + "*" * 5)
            i, l = data_generator.sample_batch('test', 100)
            pred, tls = train_step(i, l, o, optim, eval=True)
            print("Train Loss:", ls.numpy(), "Test Loss:", tls.numpy())
            pred = tf.reshape(pred, [-1, num_samples + 1, num_classes, num_classes])
            pred = tf.math.argmax(pred[:, -1, :, :], axis=2)
            l = tf.math.argmax(l[:, -1, :, :], axis=2)
            print("Test Accuracy", tf.reduce_mean(tf.cast(tf.math.equal(pred, l), tf.float32)).numpy())
            result[step] = {'train_loss':ls.numpy(), 'test_loss':tls.numpy(), 'test_acc':tf.reduce_mean(tf.cast(tf.math.equal(pred, l), tf.float32)).numpy()}
            
    return result

In [6]:
import matplotlib.pyplot as plt
results = main(num_classes=2, num_samples=1, meta_batch_size=16, random_seed=1234)
#############################
#### YOUR CODE GOES HERE ####
# pass
#############################

*****Iter 100*****
Train Loss: 5.90996 Test Loss: 215.6136
Test Accuracy 0.43
*****Iter 200*****
Train Loss: 5.3239756 Test Loss: 208.02322
Test Accuracy 0.49
*****Iter 300*****
Train Loss: 5.259412 Test Loss: 219.9459
Test Accuracy 0.465
*****Iter 400*****
Train Loss: 5.240785 Test Loss: 183.5468
Test Accuracy 0.56
*****Iter 500*****
Train Loss: 5.2426844 Test Loss: 209.20026
Test Accuracy 0.495
*****Iter 600*****
Train Loss: 5.226169 Test Loss: 183.2774
Test Accuracy 0.57
*****Iter 700*****
Train Loss: 5.2211866 Test Loss: 204.39302
Test Accuracy 0.51
*****Iter 800*****
Train Loss: 5.2181864 Test Loss: 183.43204
Test Accuracy 0.57
*****Iter 900*****
Train Loss: 5.2152643 Test Loss: 204.48862
Test Accuracy 0.51
*****Iter 1000*****
Train Loss: 5.2140813 Test Loss: 211.08575
Test Accuracy 0.49
*****Iter 1100*****
Train Loss: 5.2137294 Test Loss: 210.38808
Test Accuracy 0.49
*****Iter 1200*****
Train Loss: 5.2209496 Test Loss: 186.74167
Test Accuracy 0.55
*****Iter 1300*****
Train Loss: 

*****Iter 10200*****
Train Loss: 5.205426 Test Loss: 222.25107
Test Accuracy 0.46
*****Iter 10300*****
Train Loss: 5.205391 Test Loss: 204.68483
Test Accuracy 0.51
*****Iter 10400*****
Train Loss: 5.2053823 Test Loss: 201.17139
Test Accuracy 0.52
*****Iter 10500*****
Train Loss: 5.205382 Test Loss: 234.38223
Test Accuracy 0.425
*****Iter 10600*****
Train Loss: 5.205389 Test Loss: 197.65631
Test Accuracy 0.53
*****Iter 10700*****
Train Loss: 5.205385 Test Loss: 190.62997
Test Accuracy 0.55
*****Iter 10800*****
Train Loss: 5.2053795 Test Loss: 201.17151
Test Accuracy 0.52
*****Iter 10900*****
Train Loss: 5.2053814 Test Loss: 197.65771
Test Accuracy 0.53
*****Iter 11000*****
Train Loss: 5.2053776 Test Loss: 222.24962
Test Accuracy 0.46
*****Iter 11100*****
Train Loss: 5.2053776 Test Loss: 236.30232
Test Accuracy 0.42
*****Iter 11200*****
Train Loss: 5.2053766 Test Loss: 201.17154
Test Accuracy 0.52
*****Iter 11300*****
Train Loss: 5.2054834 Test Loss: 218.73836
Test Accuracy 0.47
*****Ite

*****Iter 20200*****
Train Loss: 5.2053633 Test Loss: 196.06392
Test Accuracy 0.535
*****Iter 20300*****
Train Loss: 5.2053633 Test Loss: 190.63174
Test Accuracy 0.55
*****Iter 20400*****
Train Loss: 5.205364 Test Loss: 203.09116
Test Accuracy 0.515
*****Iter 20500*****
Train Loss: 5.2053633 Test Loss: 201.1717
Test Accuracy 0.52
*****Iter 20600*****
Train Loss: 5.2053633 Test Loss: 208.19841
Test Accuracy 0.5
*****Iter 20700*****
Train Loss: 5.2053633 Test Loss: 211.71169
Test Accuracy 0.49
*****Iter 20800*****
Train Loss: 5.2053633 Test Loss: 180.09143
Test Accuracy 0.58
*****Iter 20900*****
Train Loss: 5.2053633 Test Loss: 225.7646
Test Accuracy 0.45
*****Iter 21000*****
Train Loss: 5.2053633 Test Loss: 204.68512
Test Accuracy 0.51
*****Iter 21100*****
Train Loss: 5.2053633 Test Loss: 190.63191
Test Accuracy 0.55
*****Iter 21200*****
Train Loss: 5.2053633 Test Loss: 215.225
Test Accuracy 0.48
*****Iter 21300*****
Train Loss: 5.2053633 Test Loss: 176.57874
Test Accuracy 0.59
*****Ite